<a href="https://colab.research.google.com/github/rubengp39/AirQuality-Raspi/blob/main/IOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [ ]:
df = pd.read_json()

In [ ]:
df.head()